In [33]:
import json

In [ ]:
data = json.load(open('/Users/georg/coding/powdr/openvm/execution_data.json'))

In [28]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_pc(runtime_values, pc):
    """
    runtime_dict: dict with key 'runtime_values' containing list of entries
    pc: program counter to filter on
    """
    entries = [e for e in runtime_values if e.get('pc') == pc]

    if not entries:
        raise ValueError(f"No entries found for pc={pc}")

    print(f"Found {len(entries)} entries for pc={pc}")

    fields = ["address_space", "address", "value"]

    for kind in ["reads", "writes"]:
        # Number of reads/writes must be the SAME across all records
        lengths = [len(e.get(kind, [])) for e in entries]
        assert(all(l == lengths[0] for l in lengths)), f"Varying number of {kind} for pc={pc}"

        n = lengths[0]
        if n == 0:
            print(f"No {kind} for pc={pc}")
            continue

        for idx in range(n):
            print(f"\nAnalyzing {kind}[{idx}]:")

            for field in fields:
                values = [e[kind][idx][field] for e in entries]
                unique_vals = pd.unique(values)

                if len(unique_vals) == 1:
                    # All the same → just print
                    print(f"  {field}: {unique_vals[0]}")
                else:
                    minimum = min(values)
                    maximum = max(values)
                    print(f"  {field}: varies (min={minimum}, max={maximum}, unique count={len(unique_vals)})")

                    bit_lengths = [value.bit_length() for value in values]

                    # Range of bins: one bin for each bit length
                    min_bits = min(bit_lengths)
                    max_bits = max(bit_lengths)

                    # Explicit bin edges: e.g. if min=4, max=7 → bins for 4,5,6,7
                    bins = range(min_bits, max_bits + 2)  # +2 so the last bin is closed properly

                    s = pd.Series(bit_lengths)
                    s.hist(bins=bins, align='left', rwidth=0.8)

                    plt.xticks(range(min_bits, max_bits + 1))  # show each bit length as a tick
                    plt.title(f"{kind}[{idx}].{field} bit lengths for pc={pc}")
                    plt.xlabel("bit length")
                    plt.ylabel("count")
                    plt.show()

In [36]:
analyze_pc(data, 2099584)

Found 1023 entries for pc=2099584

Analyzing reads[0]:
  address_space: 1
  address: 44
  value: 0

Analyzing reads[1]:
  address_space: 1
  address: 0
  value: 0
No writes for pc=2099584


/var/folders/sm/xh2t696x06zfh9q5m4xxg3y00000gn/T/ipykernel_16500/3179640590.py:33: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_vals = pd.unique(values)
